# Entrainer un détecteur d'accident sur le cloud avec YOLOv3 et Darknet 
Ce notebook montrera les étapes à suivre pour créer un détecteur d'accidents en utilisant le framework darknet et l'algorithme Yolov3. On va se profiter du GPU de la machine virtuelle offerte Google Colab pour accélérer la phase d'entrainement.

# Etape 1: Monter Darknet sur le Cloud

Les cellules suivantes permettent de cloner le repository darknet et editer le fichier de configuration Makefile pour activer OpenCV qui permettra l'excution de l'algorithme sur des videos, et le GPU pour accélérer l'entrainement de l'algorithme.

In [ ]:
# cloner darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13567 (delta 1), reused 1 (delta 0), pack-reused 13559
Receiving objects: 100% (13567/13567), 12.20 MiB | 11.74 MiB/s, done.
Resolving deltas: 100% (9254/9254), done.


In [ ]:
# activer GPU et OpenCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verifier CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# monter darknet
!make

# Etape 2: Créer des helper functions 
Ces fonctions vont être utlisées après pour simplifier l'entrainement et le test de l'algorithme

In [ ]:
#la fonction imShow prend en argument le chemin d'une image et affiche l'image
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# fonction pour uploader des fichiers sur la machine virtuelle
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# fonction pour télécharger des fichiers à partir de la machine virtuelle
def download(path):
  from google.colab import files
  files.download(path)

# Etape 3 : Monter Google Drive sur la machine virtuelle
Cette étape permet de connecter notre google drive avec la machine virtuelle de Google Colab pour ne pas perdre nos résultats avec en cas de perte de connexion avec la machine virtuelle.


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# créer un lien symbolique pour enlever les espaces et simplifier l'accès au drive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# accéder au répertoire darknet
%cd darknet


/content/darknet


In [ ]:
# Copier un fichier à partir du drive vers le répertoire darknet
!cp /mydrive/yolov3/darknet53.conv.74  ../

# Etape 4: Créer un Dataset
Pour passer à la phase d'entrainement, on aura besoin de créer un dataset avec lequel on va entrainer notre algorithme.
Il existe deux manières pour créer un dataset. 
**Méthode 1:** utiliser un outil qui automatise le téléchargement des images labellisées à partir des sites tels que : ImageNet, Open Image Dataset... 
**Méthode 2:** télécharger et labelliser manuellement à travers un outil tel que LabelImg. 

On va utiliser la deuxième méthode vu que notre dataset n'existe pas sur de tels sites. 

# Etape 5: Déplacer le Dataset vers notre Cloud VM
On va zip notre dataset et on le télécharge depuis la machine locale.

In [ ]:

!ls /mydrive/yolov3_crash

backup		   generate_train.py  obj.data	 yolov3_custom2.cfg
darknet53.conv.74  obj2.zip	      obj.names


In [ ]:
# copier le fichier .zip vers le répertoire root du cloud VM
!cp /mydrive/yolov3_crash/obj2.zip ../

In [ ]:
# unzipper le fichier
!unzip ../obj2.zip -d data/

# Etape 6: Editer les fichiers de configuration de yolov3
Editer les fichiers .cfg, obj.data, obj.names and train.txt.

## i) Fichier .cfg 

On edit le fichier .cfg pour l'adapter à ce qu'on veut faire et on le dépose sur le répertoire cfg de darknet.

In [ ]:
!cp /mydrive/yolov3_crash/yolov3_custom2.cfg ./cfg


## ii) obj.names et obj.data 
le fichier **obj.names** contient l'ensemble des classes qu'on veut detecter et qui existe sur notre dataset pour notre cas on a choisit une classe qui represente un scénario d'accident qui est le rollover.

le fichier **obj.data** contient le nombre de classe sur lequel on va entrainer notre algorithme et les chemins vers le fichier train.txt et obj.names. il contient également le chemin vers le fichier de backup où on va sauvegarder régulièrement les états d'avancement de notre entrainement afin de ne pas reprendre tout le processus en cas de suspension de l'entrainement.

In [ ]:
# deplacer les fichiers obj.names et obj.data à partir du Google Drive vers la cloud VM
!cp /mydrive/yolov3_crash/obj.names ./data
!cp /mydrive/yolov3_crash/obj.data  ./data

## iii) Générer train.txt
Ce fichier de configuration contient les chemins relatives de tous les images de notre dataset d'entrainement.
Un script python permet d'automatiser le processus de création de ce fichier

In [ ]:
# déplacer le script generate_train.py du Google Drive
!cp /mydrive/yolov3_crash/generate_train.py ./


In [ ]:
!python generate_train.py

In [ ]:
# verifier que le fichier train.txt a été bien généré
!ls data/
!cat data/train.txt | sort -n

# Etape 7: Entrainez !
Maintenant après avoir tout configurer, la denière étape consiste à executer la commande suivante de darknet.
```
!./darknet detector train <chemin vers obj.data> <chemin vers le fichier cfg> darknet53.conv.74 -dont_show
```
l'option -don't show empêche l'algorithme d'afficher le graphe montrant l'evolution de l'entrainement, vu que ça générera une erreur.  

In [ ]:

!./darknet detector train data/obj.data cfg/yolov3_custom2.cfg darknet53.conv.74 -map -dont_show

# Etape 8: Lancer la détection
On peut finalement tester notre algorithme sur des images, ou encore télécharger les poids générés après l'entrainement et tester l'algorithme en local sur des videos ou directement sur la caméra de notre machine.

In [ ]:
# modifier le fichier de configuration au mode du test 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom2.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom2.cfg
%cd ..


In [ ]:
# tester l'algorithme sur une image
!./darknet detector test data/obj.data cfg/yolov3_custom2.cfg /mydrive/yolov3/backup/yolov3_custom2_last.weights /mydrive/yolov3/images/rollovertest.jpg -thresh 0.3
imShow('predictions.jpg')